In [1]:
%pip install -U datasets
%pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset

In [3]:
# Setting model name and load dataset
data_name = "MexIvanov/CodeExercise-Python-27k-ru"
training_data = load_dataset(data_name, split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
# Converting user message to saiga prompt style
system_message = "Ты — Сайга, русскоязычный автоматический ассистент по языку программирования Python. Ты разговариваешь с людьми и помогаешь им."


def conversation_prep(data, sys_prompt=system_message):
    instr = data["instruction"]
    inp = data["input"]
    outp = data["output"]
    system_prompt = ""
    if sys_prompt != "":
        system_prompt = f"<s>system\n{system_message}</s>\n"
    if isinstance(inp, type(None)):
        inp = ""
    user_prompt = f"{instr} {inp}"
    prompt = f""
    return {"text": f"{system_prompt}<s>user\n{user_prompt}</s>\n<s>bot\n{outp}</s>\n"}

In [5]:
# Loading dataset
training_data = training_data.map(conversation_prep)
training_data = training_data.remove_columns(["instruction", "input", "output", "id"])
print(training_data)

Map:   0%|          | 0/27224 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 27224
})


In [6]:
print(training_data[0]["text"])

<s>system
Ты — Сайга, русскоязычный автоматический ассистент по языку программирования Python. Ты разговариваешь с людьми и помогаешь им.</s>
<s>user
 Напишите функцию Python с именем «add_numbers», которая принимает два параметра: «num1» и «num2» и возвращает сумму двух чисел. </s>
<s>bot
```python
def add_numbers(num1, num2):
    """
    This function takes two numbers as input and returns their sum.

    Parameters:
    num1 (int or float): The first number.
    num2 (int or float): The second number.

    Returns:
    int or float: The sum of num1 and num2.
    """
    return num1 + num2

# Example usage
result = add_numbers(5, 3)
print(result)  # Output: 8


```</s>



In [7]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")

In [8]:
from huggingface_hub import login

login(secret_value_0)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [9]:
# Push dataset on huggingface hub
training_data.push_to_hub("Megnis/python_code_instructions_27k_Saiga")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/28 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/278 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Megnis/python_code_instructions_27k_Saiga/commit/c0a39b381ba7a2efb09624f42078d2ccba856604', commit_message='Upload dataset', commit_description='', oid='c0a39b381ba7a2efb09624f42078d2ccba856604', pr_url=None, pr_revision=None, pr_num=None)